**Fundamentals of Solar Cells and Photovoltaic Systems Engineering**

**Solutions Manual - Chapter 9**

**Problem 9.7**

**Consider a PV generator composed of PV modules ~~with the electrical characteristics indicated in Table 10.3 ()~~, 'Trina_TSM_240PA05__2013_' which will be installed on a rooftop with optimal inclination and orientation.**

**The capacity of the PV generator will be chosen to match on average the current electricity consumption of the dwelling, which is 3,000kWh per year.**

**(a) Assuming a performance ratio PR=80%, estimate the number of modules required for the PV generator.**

**~~Search online for a commercial inverter with the appropriate nominal power.~~**

**(b) Selecting the inverter 'ABB__PVI_3_0_OUTD_S_US__208V_' from the SAM database, configure the PV generator by connecting the PV modules in series and parallel to ensure compatibility with the inverter. In particular, ensure that the voltage of the PV generator does not exceed the maximum input voltage of the inverter and that the maximum power point is within the specified range.**

We will use the packages [pvlib](https://pvlib-python.readthedocs.io/en/stable/), [pandas](https://pandas.pydata.org/) and [matplotlib.pyplot](https://matplotlib.org/stable/index.html) to plot the results. We will also use the package [pytz](https://pypi.org/project/pytz/) to determine the time zone of Egypt.

In [53]:
import pvlib
import pandas as pd
import matplotlib.pyplot as plt
import pytz
import math

We start by defining the location, date, and time. 

In [54]:
# El Cairo, Egypt
lat, lon =  31.233, 30.033
altitude = 23

tz = pytz.country_timezones('EG')[0]

# location
location = pvlib.location.Location(lat, lon, tz=tz)

orientation = 180 # pvlib sets orientation origin at North -> South=0

We retrieve typical meteorological year (TMY) data from PVGIS.

In [55]:
tmy, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude=lat, longitude=lon, map_variables=True)

tmy.index = tmy.index.tz_convert(tz) #use local time

We retrieve the PV modules specifications from the database at the [NREL SAM (System Advisory Monitoring)](https://sam.nrel.gov/). 

In [56]:
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

module = sandia_modules['Trina_TSM_240PA05__2013_']

module_pmp=module['Impo']*module['Vmpo']

print("Module power = " + str(round(module_pmp,1)) + " W")

Module power = 241.1 W


We calculate the Sun's coordinates and calculate the irradiance on the plane of array (POA)

In [57]:
# calculate Sun's coordinates
solar_position = location.get_solarposition(times=tmy.index)

#list of potential tilt angle
tilt = 24 #Optimal tilt calculated in Problem 9.5

# calculate irradiante at the plane of the array (poa)
poa_irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt=tilt,
                                                        surface_azimuth=orientation,
                                                        dni=tmy['dni'],
                                                        ghi=tmy['ghi'],
                                                        dhi=tmy['dhi'],
                                                        solar_zenith=solar_position['apparent_zenith'],
                                                        solar_azimuth=solar_position['azimuth'])
    
#save annual irradiation on the plane of array (POA) in kWh
annual_irradiation = 0.001*poa_irradiance['poa_global'].sum()


Assuming a PR of 80%, the number of PV modules to produce 3,000 kWh can be calculated as

In [58]:
PR=0.8
annual_energy_yield_per_module = module_pmp*(annual_irradiation/1000)*0.8
N_modules = math.ceil(3000/annual_energy_yield_per_module)
generator_capacity = N_modules*module_pmp
print("Number of modules = " + str(N_modules))
print("PV generator nominal capacity = " + str(round(generator_capacity))+' W')

Number of modules = 8
PV generator nominal capacity = 1929 W


We retrieve the inverter data from the SAM database.

In [59]:
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
inverter = sapm_inverters['ABB__PVI_3_0_OUTD_S_US__208V_']

The maximum number of PV modules that can be conected in series is determined by the inverter input voltage

In [60]:
modules_in_series = inverter['Vdcmax']/module['Vmpo']
modules_in_series

16.514140232574142

Hence, the 8 PV modules can be conected in series. 

The voltage produced by the 8 series-connected PV modules is within the maximum and minimum voltage acepted by the inverter. 

In [61]:
N_modules*module['Vmpo']

232.528

In [62]:
inverter['Mppt_high']

480.0

In [63]:
inverter['Mppt_low']

100.0